In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

REMOTE_RTS_DATA = "/media/sinergia/RTS/"
REMOTE_VIDEOS = '/mnt/rts/'

LOCAL_RTS_DATA = "/media/data/rts/"
METADATA = LOCAL_RTS_DATA + 'metadata'
LOCAL_VIDEOS = LOCAL_RTS_DATA + 'archive'

OUTDIR = 'data'


/home/kirell/work/rts


In [2]:
from datetime import datetime
import orjson
import pandas as pd
import zipfile
import os
import io
from typing import Dict, List, Optional, Tuple
from pathlib import Path

In [3]:
# LOCAL imports
import rts
import rts.pipeline
import rts.utils
import rts.io.media
import rts.features.audio
import rts.features.text
import rts.features.image

LOG = rts.utils.get_logger()

/home/kirell/.cache/pypoetry/virtualenvs/rts-506MfBOj-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# vidx = rts.metadata.read_video_folder_index(os.path.join(OUTDIR, 'vidx.json'))
# df = rts.metadata.read_all_metadata(METADATA, vidx)
# rts.metadata.metadata_to_hdf5(OUTDIR, 'rts_metadata', df)

df = rts.metadata.load_metadata_hdf5(METADATA, 'rts_metadata')
# sample_df = rts.metadata.get_one_percent_sample(df)
# # rts.metadata.export_metadata_stats(sample_df, Path(OUTDIR) / 'sample')
# # rts.metadata.export_metadata_stats(df, OUTDIR)
# sample_df.head()

# medias = rts.metadata.load_all_media_info(LOCAL_VIDEOS)
# clips = rts.metadata.load_all_clips(LOCAL_VIDEOS)

# clip_df = rts.metadata.build_clips_df(LOCAL_VIDEOS, METADATA)
# rts.utils.dataframe_to_hdf5(METADATA, 'clips', clip_df)

,guid,mediaFolderPath,mediaDuration,ratio,formatResolution,publishedDate,categoryName,assetType,contentType,backgoundType,collection,publishedBy,rights,title,resume,geoTheme,resumeSequence,created,published,sequences
mediaId,,,,,,,,,,,,,,,,,,,,
ZE002100,AA1101000770,/mnt/rts/0/0/1/ZE002100,1524,16:9,SD,2010-04-29T17:40:00Z,Programme,Programme,Emissions de conseil,None,Dolce vita,TSR 1,Restriction/Condition,CAMPING - 10.04.29,"* Plateau : 20100429, SIKI Muriel reçoit ses i...","[LIEU INDÉTERMINÉ, VAUD, PLATEAU]",[00:01.24\r\nPlateau : diverses présentations ...,1670505072,1272562800,"{'890511': {'guid': 'AA1101000770', 'mediaId':..."
ZS009400,AA1106004840,/mnt/rts/0/0/4/ZS009400,530,16:9,SD,2011-03-05T00:00:00Z,Programme,Programme,Autres émissions religieuses,None,Faut pas croire,TSR 1,Restriction/Condition,11.03.05,"* Plateau : 20110305, BOISSET Christophe reçoi...","[LIEU INDÉTERMINÉ, NEUVEVILLE, LA, PLATEAU]",[00:01.18\r\nPlateau : diverses présentations ...,1670505072,1299283200,"{'897154': {'guid': 'AA1106004840', 'mediaId':..."
ZM001400,AA1201021870,/mnt/rts/0/0/4/ZM001400,43,16:9,SD,2012-01-22T00:00:00Z,Programme,Programme,"Bandes-annonces, lancements",None,Bande Annonce,TSR 2,Inconnu,Alain Delon. 2BLS CINEMA 2201,None,None,None,1670505073,1327190400,{}
ZT002200,AA1202023530,/mnt/rts/0/0/2/ZT002200,4445,16:9,SD,2012-04-08T00:00:00Z,Programme,Retransmission sportive,Retransmissions en direct,None,Motocyclisme,TSR 2,Restriction/Condition,MOTO3 - GRAND PRIX DU QATAR LOSAIL 2012,None,None,None,1670505073,1333843200,{}
ZM002200,AA1203026020,/mnt/rts/0/0/2/ZM002200,60,16:9,HD,2012-04-07T00:00:00Z,Programme,Programme,"Bandes-annonces, lancements",None,Bande Annonce,None,Inconnu,BLS SAMEDI 0704,None,None,None,1670505073,1333756800,{}


In [19]:
sujets = rts.metadata.filter_by_asset_type(df, nested_struct='0/0')

In [20]:
sujets

,guid,mediaFolderPath,mediaDuration,ratio,formatResolution,publishedDate,categoryName,assetType,contentType,backgoundType,collection,publishedBy,rights,title,resume,geoTheme,resumeSequence,created,published,sequences
mediaId,,,,,,,,,,,,,,,,,,,,
ZB027000,GE9702459644,/mnt/rts/0/0/0/ZB027000,70,4:3,SD,1997-02-13T00:00:00Z,Sujet,Sujet journal télévisé,Actualités régionales,None,Vaud région,S4,Inconnu,19H05 - 97.02.13,"* Lausanne : 19970213, Une défaillance d'un ai...",[LAUSANNE],"[17:59.12\r\nLausanne : itw SAUTEREL, Alain, r...",1670505322,855792000,"{'1048260': {'guid': 'GE9702459644', 'mediaId'..."
ZB045000,GE9804357740,/mnt/rts/0/0/0/ZB045000,71,4:3,SD,1998-04-18T00:00:00Z,Sujet,Sujet journal télévisé,"Téléjournal, flashes",None,Le téléjournal,TSR 1,Inconnu,98.04.18,"* Mogadiscio : 19980418, préprod à base de New...",[MOGADISCIO],"[00:21.04\r\nMogadiscio : zoom arrière, ext im...",1670505126,892857600,"{'1073159': {'guid': 'GE9804357740', 'mediaId'..."
ZB053000,GE0312298610,/mnt/rts/0/0/0/ZB053000,90,4:3,SD,2003-12-04T00:00:00Z,Sujet,Sujet journal télévisé,Actualités régionales,None,le 19:00 des régions,TSR 1,Inconnu,03.12.04,"* Peseux, Neuchâtel-ville : 20031204, le canto...","[NEUCHATEL-VILLE, PESEUX]","[18:42.43\r\nPeseux, diverses séquences : URFE...",1670505094,1070496000,"{'1168738': {'guid': 'GE0312298610', 'mediaId'..."
ZB056000,GE9705374460,/mnt/rts/0/0/0/ZB056000,124,4:3,SD,1997-05-19T00:00:00Z,Sujet,Sujet journal télévisé,"Téléjournal, flashes",None,Le téléjournal,TSR,Restriction/Condition,97.05.19,"* Cannes : 19970518, le jury du Festival inter...",[CANNES],"[00:18.06\r\nCannes : plan taille ADJANI, Isab...",1670505125,864000000,"{'1054894': {'guid': 'GE9705374460', 'mediaId'..."
ZB057000,GE8405283593,/mnt/rts/0/0/0/ZB057000,128,4:3,SD,1983-03-10T00:00:00Z,Sujet,Sujet journal télévisé,Actualités régionales,None,Journal romand,TSR,Restriction/Condition,TEMOIN - 83.03.10,None,"[VAUD, MONTREUX, SUISSE]","[Montreux : pour la premiere fois en Suisse, d...",1670505263,416102400,"{'917079': {'guid': 'GE8405283593', 'mediaId':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZU149900,AA2110255575,/mnt/rts/0/0/9/ZU149900,163,16:9,HD,2021-10-30T00:00:00Z,Sujet,Sujet journal télévisé,"Téléjournal, flashes",None,19h30,TSR 1,Inconnu,FRIGO CIMETIERE PESTIFERES_RTS000002636535,"Le cimetière des pestiférés du Boéchet, dans l...","[BOIS, LES]",None,1670505341,1635552000,{}
ZU150900,AA2111262531,/mnt/rts/0/0/9/ZU150900,137,16:9,HD,2021-11-21T00:00:00Z,Sujet,Sujet journal télévisé,None,None,19h30,TSR 1,Inconnu,SO JU PREVENTION FEMINICIDES_RTS000002657106,None,None,None,1670505144,1637452800,{}
ZU151900,AA2112269440,/mnt/rts/0/0/9/ZU151900,99,16:9,HD,2021-12-13T00:00:00Z,Sujet,Sujet journal télévisé,None,None,12h45,TSR 1,Inconnu,MI VD TRANSGENRES ECOLES_RTS000002675504,None,None,None,1670505088,1639353600,{}


In [23]:
sujets.mediaDuration.mean()

132.0706620773436

In [4]:
rts.features.image.embed_images([Path(OUTDIR) / 'testori1.jpg', 
                                 Path(OUTDIR) / 'testori2.jpg'])

INFO:RTS:Load model: resnet50


array([[0.06276648, 1.1780145 , 0.3374293 , ..., 0.2396185 , 0.16062912,
        0.41618043],
       [0.28102228, 0.5253164 , 0.09940885, ..., 0.00276155, 0.019754  ,
        0.05477377]], dtype=float32)

In [18]:
video_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000.mp4')

outpath = Path(OUTDIR) / 'test_trim.mp4'
outpath = rts.io.media.trim(video_path,  Path(OUTDIR) / 'test_trim.mp4', 300, 310)


Args: ['-hide_banner', '-loglevel', 'error', '-i', '/media/data/rts/archive/0/0/0/ZB001000/ZB001000.mp4', '-filter_complex', '[0:v]trim=end=310:start=300[s0];[s0]setpts=PTS-STARTPTS[s1];[0:a]atrim=end=310:start=300[s2];[s2]asetpts=PTS-STARTPTS[s3];[s1][s3]concat=a=1:n=1:v=1[s4][s5]', '-map', '[s4]', '-map', '[s5]', '-movflags', 'faststart', 'data/test_trim.mp4', '-y']


In [ ]:
client = rts.db.fulltext.get_client()
midx = rts.db.fulltext.get_or_create_media_index(client)
# Fulltext Index from HDF5
# midx.delete_all_documents()
rts.db.fulltext.index_media(df, midx)
# Wait 30 sec for full indexation
midx.get_stats().number_of_documents  # 522103

In [ ]:
medias = rts.metadata.load_all_media_info(LOCAL_VIDEOS)

In [ ]:
clips = rts.metadata.load_all_clips(LOCAL_VIDEOS)

In [ ]:
df = rts.metadata.build_clips_df(LOCAL_VIDEOS)

In [ ]:
atlases = rts.metadata.create_clip_texture_atlases(df, LOCAL_RTS_DATA,
                                                   '0', # folder name
                                                   tile_size=64,
                                                   flip=True,
                                                   no_border=True, 
                                                   format='jpg')

In [ ]:
import PIL
import os
from PIL import ImageOps, Image

In [ ]:
test_im_path = os.path.join(OUTDIR, 'test_image.jpg')
im = Image.open(test_im_path)
# rts.io.media.save_image_pyramid(im, Path(OUTDIR) / 'test_image_pyramid', 'test_image', True)

In [ ]:
video_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000.mp4')
# audio_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000_audio.aac')
# rts.io.media.get_media_info(video_path)

# scenes = rts.io.media.detect_scenes(video_path)
# scenes = rts.io.media.filter_scenes(scenes, 5)
# clips = rts.io.media.save_scenes_images(scenes, video_path, OUTDIR)
# rts.pipeline.save_scenes(clips, OUTDIR)

In [5]:
test_video_folder = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000')
audio_path = os.path.join(test_video_folder, 'ZB001000_audio.m4a')

# outpath = rts.io.media.trim_media(video_path, 30, 40, Path(OUTDIR) / 'test_trim.mp4')
# rts.io.media.get_media_info(outpath)

In [ ]:
clips = rts.pipeline.extract_location_clips_from_transcript(transcript,
    25,
    OUTDIR,
    video_path,
    min_seconds=3,
    extend_duration=6,
    num_images=3,
    force=True
)

In [ ]:
imgs = list(rts.utils.recursive_glob(Path(OUTDIR) / 'images' / 'original', '.jpg'))
imgs

In [ ]:
rts.io.media.create_square_atlases(
    imgs,
    Path(OUTDIR) / 'atlas',
    width=512,
    max_tile_size=128,
    no_border=True,
    flip=True,
    atlas_prefix='small_atlas',
    bg_color=(255, 255, 255, 255)
)

In [ ]:
res = rts.features.text.enrich_scenes_with_transcript(clips.copy(), t)
# res

In [ ]:
swiss_cities = rts.features.text.get_swiss_cities()

In [ ]:
input_video_folder = os.path.join(REMOTE_VIDEOS, '0', '0', '0', 'ZB001000')
# input_video_folder = os.path.join(VIDEOS, '0', '0', 'ZB002000')
# v, a = rts.io.archive.get_raw_video_audio_parts(input_video_folder)

# '/'.join(input_video_folder.split('/')[-4:])
# remuxed = rts.pipeline.create_optimized_media(input_video_folder, LOCAL_VIDEOS, force=True)
# print(remuxed)

rts.pipeline.process_media(input_video_folder, LOCAL_VIDEOS, 
    compute_transcript=True, force_scene=False, force_trans=False)

In [ ]:
%ls {remux_folder}

In [ ]:
fts.groupby('mediaId').count().sort_values('location', ascending=False)

In [ ]:
fts

In [ ]:
swiss_cities = rts.features.text.get_swiss_cities()

In [ ]:
rts.utils.obj_to_json(r, os.path.join(OUTDIR, 'transcript.json'))

In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.extract_audio(a, os.path.join(OUTDIR, 'myaudio'))